### LSTM Model

In [1]:
# restart kernel after running this cell
!pip install pandas --upgrade

    100% |████████████████████████████████| 10.1MB 2.8MB/s 
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime
import os
import glob
import re
import math
import json
import random

from IPython import display
from matplotlib import cm

import tensorflow as tf
from tensorflow.python.data import Dataset

from google.colab import drive
drive.mount('/content/drive')

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from itertools import chain
import time
import multiprocessing
import copy
import pickle
import datetime as dt
import h5py
import re
from subprocess import call
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.models import load_model
from keras import Model
from keras import backend as K
from keras.utils import to_categorical
import gc
from keras.layers import Input, LSTM, Bidirectional, Dense, Embedding
%matplotlib inline

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [2]:
%%time
combined_data = pd.read_pickle('/content/drive/My Drive/BT4222 Group/workspace/data/combined.p')

CPU times: user 2.1 s, sys: 2.98 s, total: 5.08 s
Wall time: 16.9 s


In [0]:
combined_data.shape

In [0]:
combined_data.columns

In [0]:
combined_data.head()

In [0]:
combined_data['assetCode'].unique().shape

In [0]:
features_list = [
    'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'open_close', 'oc_average', 'turnover',]

In [0]:
second_level_features = ['open_close_relative', 'turnover_relative', 'volume_relative', 'returnsOpenPrevMktres1_relative', 'returnsOpenPrevMktres10_relative',]
features_list += second_level_features

In [0]:
def reshape_data(data, sequence_len=21, features_list=features_list[:-len(second_level_features)]):
  data_list, lgbm_list, target_list, returns_list, date_list = [], [], [], [], []

  for assetCode, asset_data in data.groupby('assetCode'):
    # binarize the target variable
    returns = asset_data['returnsOpenNextMktres10']
    target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
    dates = asset_data.index
    asset_data = asset_data[features_list]
    
    # fill missing values with mean
    ini_array = asset_data.values
    col_mean = np.nanmean(ini_array, axis = 0) 
    inds = np.where(np.isnan(ini_array)) 
    ini_array[inds] = np.take(col_mean, inds[1])
    
    # safety net
    ini_array = ini_array[~(np.isnan(ini_array).sum(axis=1) > 0)]
    
    # reshape data for LSTM
    reshaped_data = np.repeat(ini_array[:, np.newaxis, :], sequence_len, axis=1)
    for i in range(reshaped_data.shape[1]):
      reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

    # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
    reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
    ini_array = ini_array[sequence_len-1:, :]
    target = target.values[sequence_len-1:]
    returns = returns.values[sequence_len-1:]
    dates = dates[sequence_len-1:]

    data_list.append(reshaped_data)
    lgbm_list.append(ini_array)
    target_list.append(target)
    returns_list.append(returns)
    date_list.append(dates)
  
  reshaped_data, lgbm_data, target, returns, dates = np.concatenate(data_list), np.concatenate(lgbm_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)
  
  return reshaped_data, lgbm_data, target, returns, dates


In [0]:
lstm_params = {
    'batch_size': 252,
    'epochs': 2,
  }

def create_model(stateful=False, sequence_len=21, batch_size=lstm_params['batch_size']):
    model = Sequential()
    model.add(LSTM(32,
              input_shape=(sequence_len, len(features_list)-len(second_level_features)),
              batch_size=batch_size,
              stateful=stateful,
              kernel_initializer='glorot_normal',
              bias_initializer='zeros'))
    model.add(Dense(1,
              activation='sigmoid',
              kernel_initializer='glorot_normal',
              bias_initializer='zeros'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
import lightgbm as lgb

#Set model params
params = {}
params['learning_rate'] = 0.5            # 0.003 to 0.1 gives significant increase in sigma
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 30                # 10 to 30 gives significant increase in sigma
params['min_data'] = 20
params['max_depth'] = 20

In [0]:
def get_sigma(scaled_scores, test_returns, test_dates):
  '''
  scaled_scores: numpy array
  test_returns: numpy array
  test_dates: numpy array
  '''

  grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

  day_scores = []
  for ts in grouped_dict:
    grouped_index = grouped_dict[ts]
    day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
    day_scores.append(day_score)

  return np.mean(day_scores) / np.std(day_scores, ddof=1)


In [0]:
def reshape_LSTM_stack(data, sequence_len=21):
  # reshape data for LSTM
  reshaped_data = np.repeat(data[:, np.newaxis, :], sequence_len, axis=1)
  for i in range(reshaped_data.shape[1]):
    reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

  # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
  reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]

  return reshaped_data

def reshape_LSTM_y(data, sequence_len=21):
  return data[sequence_len-1:]


def create_stack_model(stateful=False, sequence_len=21, batch_size=lstm_params['batch_size']):
    model = Sequential()
    model.add(LSTM(32,
              input_shape=(sequence_len, len(second_level_features)+2),  ## need to change second parameter
              batch_size=batch_size,
              stateful=stateful,
              kernel_initializer='glorot_normal',
              bias_initializer='zeros'))
    model.add(Dense(1,
              activation='sigmoid',
              kernel_initializer='glorot_normal',
              bias_initializer='zeros'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# array of start dates from Jan 2011-Jan 2016
# array of end dates from Dec 2011-Dec 2016
start_dates = ['2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01']
train_end_dates = ['2011-12-31','2012-12-31','2013-12-31','2014-12-31','2015-12-31']
val_end_dates = ['2012-06-30','2013-06-30','2014-06-30','2015-06-30','2016-06-30']
stack_start_dates = ['2012-07-01','2013-07-01','2014-07-01','2015-07-01','2016-07-01']
stack_end_dates = ['2012-12-31','2013-12-31','2014-12-31','2015-12-31','2016-12-31']

# store prediction scores
sigma_score = np.empty(0,)
lstm_sigma = np.empty(0,)
lgbm_sigma = np.empty(0,)
accuracy_score = np.empty(0,)
confusion_matrix = np.empty(0,)

for item in range (0,5): 
    
  # period for base model
  train_start, train_end = pd.to_datetime(start_dates[item]), pd.to_datetime(train_end_dates[item])
  temp = combined_data.set_index('time')[train_start: train_end]

  n = int(len(temp)/6)  #chunk row size
               
  # period for validation model and stack model                                       
  val_start, val_end = pd.to_datetime(start_dates[item+1]), pd.to_datetime(val_end_dates[item])
  validation_set = combined_data.set_index('time')[val_start: val_end]
  stack_start, stack_end = pd.to_datetime(stack_start_dates[item]), pd.to_datetime(stack_end_dates[item])
  stack_set = combined_data.set_index('time')[stack_start: stack_end]
                                          
  stack_train_features = np.empty(shape = (0,(2+len(second_level_features))))
  stack_val_features = np.empty(shape = (0,(2+len(second_level_features)))) 
  stack_test_features = np.empty(shape = (0,(2+len(second_level_features))))   
  stack_train_y = np.empty(shape = (0,)) 
  
  for i in range(0,6):
      
    if i != 5:
      ## get predictions as features for stack model 
      train_data = temp[0:n*(i+1)] 
      validation_data = validation_set  
      test_data = temp[n*(i+1):n*(i+2)]
                                          
    else:
      ## get new features for stack test
      train_data = temp
      validation_data = validation_set
      test_data = stack_set

    train_data[features_list] = scaler.fit_transform(train_data[features_list])
    validation_data[features_list] = scaler.transform(validation_data[features_list])
    test_data[features_list] = scaler.transform(test_data[features_list])

    # sampling

    stocks = train_data['assetCode'].unique()

    # N = min(100, stocks.shape[0])
    N = stocks.shape[0]

    np.random.seed(1)
    selected_stocks = set(np.random.choice(stocks, N))
    train_data = train_data[train_data['assetCode'].map(lambda x: x in selected_stocks)]
    validation_data = validation_data[validation_data['assetCode'].map(lambda x: x in selected_stocks)]
    test_data = test_data[test_data['assetCode'].map(lambda x: x in selected_stocks)]

    # reshape data                                         
    x_train, lgbm_train, y_train, train_returns, train_dates = reshape_data(train_data)
    x_val, lgbm_val, y_val, val_returns, val_dates = reshape_data(validation_data)
    x_test, lgbm_test, y_test, test_returns, test_dates = reshape_data(test_data)

    # tweak to match with batch_size
    train_until = x_train.shape[0] - x_train.shape[0] % lstm_params['batch_size']
    val_until = x_val.shape[0] - x_val.shape[0] % lstm_params['batch_size']
    test_until = x_test.shape[0] - x_test.shape[0] % lstm_params['batch_size']

    x_train = x_train[:train_until]
    y_train = y_train[:train_until]
    lgbm_train = lgbm_train[:train_until]
    train_returns = train_returns[:train_until]
    train_dates = train_dates[:train_until]
    x_val = x_val[:val_until]
    y_val = y_val[:val_until]
    lgbm_val = lgbm_val[:val_until]
    val_returns = val_returns[:val_until]
    val_dates = val_dates[:val_until]
    x_test = x_test[:test_until]
    y_test = y_test[:test_until]
    lgbm_test = lgbm_test[:test_until]
    test_returns = test_returns[:test_until]
    test_dates = test_dates[:test_until]

    np.random.seed(1)
    train_shuffle = np.random.permutation(range(x_train.shape[0]))
    lgbm_train_shuffle = np.random.permutation(range(lgbm_train.shape[0]))
    val_shuffle = np.random.permutation(range(x_val.shape[0]))
    lgbm_val_shuffle = np.random.permutation(range(lgbm_val.shape[0]))
    test_shuffle = np.random.permutation(range(x_test.shape[0]))
    lgbm_test_shuffle = np.random.permutation(range(lgbm_test.shape[0]))
    x_train, lgbm_train, y_train, train_returns, train_dates = x_train[train_shuffle], lgbm_train[lgbm_train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
    x_val, lgbm_val, y_val, val_returns, val_dates = x_val[val_shuffle], lgbm_val[lgbm_val_shuffle], y_val[val_shuffle], val_returns[val_shuffle], val_dates[val_shuffle]
    x_test, lgbm_test, y_test, test_returns, test_dates = x_test[test_shuffle], lgbm_test[lgbm_test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]


    lstm_model = create_model()

    if second_level_features == []:
      
      # LGBM model
      
      d_train = lgb.Dataset(lgbm_train, label=y_train)

      lgbmodel = lgb.train(params, d_train, 100)
      
      scaled_val_scores_lgbm = lgbmodel.predict(lgbm_val, batch_size=252).reshape(-1) * 2 - 1
      scaled_scores_lgbm = lgbmodel.predict(lgbm_test, batch_size=252).reshape(-1) * 2 - 1
      lgbm_sigma = np.append(lgbm_sigma, get_sigma(scaled_scores_lgbm, test_returns, test_dates))
      
      # LSTM model
      
      if i != 5:
      
        lstm_model.fit(x_train,
                     y_train,
                     batch_size=lstm_params['batch_size'],
                     epochs= lstm_params['epochs'],
                     verbose=1,
                     validation_data=(x_test, y_test),
                     shuffle=False)

        scaled_scores_lstm = lstm_model.predict(x_test, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
        lstm_sigma = np.append(lstm_sigma, get_sigma(scaled_scores_lstm, test_returns, test_dates))

        curr_stack_features = np.concatenate([scaled_scores_lstm.reshape(-1,1), scaled_scores_lgbm.reshape(-1,1)], axis=1)
        stack_train_features = np.concatenate([stack_train_features,curr_stack_features], axis=0)
        stack_train_y = np.concatenate([stack_train_y,y_test], axis=0)

      else:
        
        lstm_model.fit(x_train,
                     y_train,
                     batch_size=lstm_params['batch_size'],
                     epochs= lstm_params['epochs'],
                     verbose=1,
                     validation_data=(x_val, y_val),
                     shuffle=False)
        
        scaled_val_scores_lstm = lstm_model.predict(x_val, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
        scaled_scores_lstm = lstm_model.predict(x_test, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
        lstm_sigma = np.append(lstm_sigma, get_sigma(scaled_val_scores_lstm, val_returns, val_dates))

        curr_stack_features = np.concatenate([scaled_scores_lstm.reshape(-1,1), scaled_scores_lgbm.reshape(-1,1)], axis=1)
        stack_test_features = np.concatenate([stack_test_features,curr_stack_features], axis=0)
        curr_val_features = np.concatenate([scaled_val_scores_lstm.reshape(-1,1), scaled_val_scores_lgbm.reshape(-1,1)], axis=1)
        stack_val_features = np.concatenate([stack_val_features,curr_val_features], axis=0)
        
    else:
      
      # LGBM model

      d_train = lgb.Dataset(lgbm_train, label=y_train)

      lgbmodel = lgb.train(params, d_train, 100)
      
      scaled_val_scores_lgbm = lgbmodel.predict(lgbm_val, batch_size=252).reshape(-1) * 2 - 1
      scaled_scores_lgbm = lgbmodel.predict(lgbm_test, batch_size=252).reshape(-1) * 2 - 1
      lgbm_sigma = np.append(lgbm_sigma, get_sigma(scaled_scores_lgbm, test_returns, test_dates))
      
      # LSTM model
      
      if i != 5:

        lstm_model.fit(x_train,
               y_train,
               batch_size=lstm_params['batch_size'],
               epochs=lstm_params['epochs'],
               verbose=1,
               validation_data=(x_test, y_test),
               shuffle=False)

        scaled_scores_lstm = lstm_model.predict(x_test, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
        lstm_sigma = np.append(lstm_sigma, get_sigma(scaled_scores_lstm, test_returns, test_dates))
      
        curr_stack_features = np.concatenate([x_test[:,0,(x_test.shape[2]-len(second_level_features)):(x_test.shape[2])], scaled_scores_lstm.reshape(-1,1), scaled_scores_lgbm.reshape(-1,1)], axis=1)
        stack_train_features = np.concatenate([stack_train_features,curr_stack_features], axis=0)
        stack_train_y = np.concatenate([stack_train_y,y_test], axis=0)

      else:
        
        lstm_model.fit(x_train,
               y_train,
               batch_size=lstm_params['batch_size'],
               epochs=lstm_params['epochs'],
               verbose=1,
               validation_data=(x_val, y_val),
               shuffle=False)
        
        scaled_val_scores_lstm = lstm_model.predict(x_val, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
        lstm_sigma = np.append(lstm_sigma, get_sigma(scaled_val_scores_lstm, val_returns, val_dates))
        #scaled_scores_lstm = lstm_model.predict(x_test, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
      
        #curr_stack_features = np.concatenate([x_test[:,0,(x_test.shape[2]-len(second_level_features)):(x_test.shape[2])], scaled_scores_lstm.reshape(-1,1), scaled_scores_lgbm.reshape(-1,1)], axis=1)
        #stack_test_features = np.concatenate([stack_test_features,curr_stack_features], axis=0)
        curr_val_features = np.concatenate([x_val[:,0,(x_val.shape[2]-len(second_level_features)):(x_val.shape[2])], scaled_val_scores_lstm.reshape(-1,1), scaled_val_scores_lgbm.reshape(-1,1)], axis=1)
        stack_val_features = np.concatenate([stack_val_features,curr_val_features], axis=0)
             
        
  stack_train_features = reshape_LSTM_stack(stack_train_features)
  stack_val_features = reshape_LSTM_stack(stack_val_features)
  #stack_test_features = reshape_LSTM_stack(stack_test_features)
  stack_train_y = reshape_LSTM_y(stack_train_y)
  val_returns = reshape_LSTM_y(val_returns)
  val_dates = reshape_LSTM_y(val_dates)
  #test_returns = reshape_LSTM_y(test_returns)
  #test_dates = reshape_LSTM_y(test_dates)

  # tweak to match with batch_size
  train_until = stack_train_features.shape[0] - stack_train_features.shape[0] % lstm_params['batch_size']
  val_until = stack_val_features.shape[0] - stack_val_features.shape[0] % lstm_params['batch_size']
  #test_until = stack_test_features.shape[0] - stack_test_features.shape[0] % lstm_params['batch_size']

  stack_train_features = stack_train_features[:train_until]
  stack_train_y = stack_train_y[:train_until]
  stack_val_features = stack_val_features[:val_until]
  val_returns = val_returns[:val_until]
  val_dates = val_dates[:val_until]
  #stack_test_features = stack_test_features[:test_until]
  #test_returns = test_returns[:test_until]
  #test_dates = test_dates[:test_until]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(stack_train_features.shape[0]))
  val_shuffle = np.random.permutation(range(stack_val_features.shape[0]))
  #test_shuffle = np.random.permutation(range(stack_test_features.shape[0]))
  stack_train_features, stack_train_y = stack_train_features[train_shuffle], stack_train_y[train_shuffle]
  stack_val_features, val_returns, val_dates = stack_val_features[val_shuffle], val_returns[val_shuffle], val_dates[val_shuffle]
  #stack_test_features, test_returns, test_dates = stack_test_features[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  lstm_stack_model = create_stack_model()

  lstm_stack_model.fit(stack_train_features,
               stack_train_y,
               batch_size=lstm_params['batch_size'],
               epochs=lstm_params['epochs'],
               verbose=1,
               validation_data=(stack_val_features, val_returns),
               shuffle=False)

  #Prediction
  
  scaled_scores_lstm = lstm_stack_model.predict(stack_val_features, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
  #scaled_scores_lstm = lstm_stack_model.predict(stack_test_features, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1

  curr_sigma_score = get_sigma(scaled_scores_lstm, val_returns, val_dates)
  sigma_score = np.append(sigma_score, curr_sigma_score)

  classify = lambda x: 1 if x > 0 else 0
  scaled_scores_binary = np.array([classify(i) for i in scaled_scores_lstm])
  val_returns_binary = np.array([classify(i) for i in val_returns])
  curr_accuracy_score = metrics.accuracy_score(val_returns_binary, scaled_scores_binary)
  curr_confusion_matrix = metrics.confusion_matrix(val_returns_binary, scaled_scores_binary)
  accuracy_score = np.append(accuracy_score, curr_accuracy_score)
  confusion_matrix = np.append(confusion_matrix, curr_confusion_matrix)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 22932 samples, validate on 22680 samples
Epoch 1/2
22932/22932 [==============================] - 4s 155us/step - loss: 0.6673 - acc: 0.5910 - val_loss: 0.7453 - val_acc: 0.5039
Epoch 2/2
22932/22932 [==============================] - 2s 109us/step - loss: 0.6521 - acc: 0.6195 - val_loss: 0.7560 - val_acc: 0.4987


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

Train on 65772 samples, validate on 22428 samples
Epoch 1/2
65772/65772 [==============================] - 7s 113us/step - loss: 0.6764 - acc: 0.5751 - val_loss: 0.6842 - val_acc: 0.5607
Epoch 2/2
65772/65772 [==============================] - 6s 95us/step - loss: 0.6659 - acc: 0.5930 - val_loss: 0.6817 - val_acc: 0.5691


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

...

Train on 156996 samples, validate on 22428 samples
Epoch 1/2
156996/156996 [==============================] - 20s 127us/step - loss: 0.6788 - acc: 0.5683 - val_loss: 0.7274 - val_acc: 0.5076
Epoch 2/2
156996/156996 [==============================] - 14s 91us/step - loss: 0.6710 - acc: 0.5819 - val_loss: 0.7313 - val_acc: 0.4959


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

Train on 201600 samples, validate on 22176 samples
Epoch 1/2
201600/201600 [==============================] - 24s 119us/step - loss: 0.6778 - acc: 0.5675 - val_loss: 0.6880 - val_acc: 0.5726
Epoch 2/2
201600/201600 [==============================] - 18s 90us/step - loss: 0.6636 - acc: 0.5939 - val_loss: 0.6976 - val_acc: 0.5508


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]


Train on 245952 samples, validate on 114660 samples
Epoch 1/2
245952/245952 [==============================] - 31s 124us/step - loss: 0.6780 - acc: 0.5692 - val_loss: 0.7345 - val_acc: 0.4991
Epoch 2/2
245952/245952 [==============================] - 24s 98us/step - loss: 0.6666 - acc: 0.5901 - val_loss: 0.7297 - val_acc: 0.4971
Train on 113652 samples, validate on 114408 samples
Epoch 1/2
113652/113652 [==============================] - 19s 164us/step - loss: 0.6890 - acc: 0.5403 - val_loss: 0.5845 - val_acc: 0.0000e+00
Epoch 2/2
113652/113652 [==============================] - 13s 110us/step - loss: 0.6875 - acc: 0.5438 - val_loss: 0.6405 - val_acc: 0.0000e+00


In [11]:
sigma_score

array([ 0.3962073 ,  0.15815547,  0.17446199,  0.10734467, -0.22639669])

In [12]:
accuracy_score

array([0.53579852, 0.50545987, 0.50256769, 0.49953033, 0.47877771])

In [13]:
confusion_matrix

array([5.7194e+04, 7.5000e+01, 4.9524e+04, 5.5000e+01, 2.8510e+04,
       2.5719e+04, 2.6997e+04, 2.5370e+04, 2.8485e+04, 2.4067e+04,
       2.7077e+04, 2.3187e+04, 2.5357e+04, 2.5883e+04, 2.6330e+04,
       2.6758e+04, 5.3421e+04, 1.2110e+03, 5.8421e+04, 1.3550e+03])

### Reference Code

In [0]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM

# ----------------------------------------------------------
# EDITABLE PARAMETERS
# Read the documentation in the script head for more details
# ----------------------------------------------------------

# length of input
input_len = 1000

# The window length of the moving average used to generate
# the output from the input in the input/output pair used
# to train the LSTM
# e.g. if tsteps=2 and input=[1, 2, 3, 4, 5],
#      then output=[1.5, 2.5, 3.5, 4.5]
tsteps = 2

# The input sequence length that the LSTM is trained on for each output point
lahead = 10

# training parameters passed to "model.fit(...)"
batch_size = 1
epochs = 10

# ------------
# MAIN PROGRAM
# ------------

print("*" * 33)
if lahead >= tsteps:
    print("STATELESS LSTM WILL ALSO CONVERGE")
else:
    print("STATELESS LSTM WILL NOT CONVERGE")
print("*" * 33)

np.random.seed(1986)

print('Generating Data...')


def gen_uniform_amp(amp=1, xn=10000):
    """Generates uniform random data between
    -amp and +amp
    and of length xn

    Arguments:
        amp: maximum/minimum range of uniform data
        xn: length of series
    """
    data_input = np.random.uniform(-1 * amp, +1 * amp, xn)
    data_input = pd.DataFrame(data_input)
    return data_input

# Since the output is a moving average of the input,
# the first few points of output will be NaN
# and will be dropped from the generated data
# before training the LSTM.
# Also, when lahead > 1,
# the preprocessing step later of "rolling window view"
# will also cause some points to be lost.
# For aesthetic reasons,
# in order to maintain generated data length = input_len after pre-processing,
# add a few points to account for the values that will be lost.
to_drop = max(tsteps - 1, lahead - 1)
data_input = gen_uniform_amp(amp=0.1, xn=input_len + to_drop)

# set the target to be a N-point average of the input
expected_output = data_input.rolling(window=tsteps, center=False).mean()

# when lahead > 1, need to convert the input to "rolling window view"
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.repeat.html
if lahead > 1:
    data_input = np.repeat(data_input.values, repeats=lahead, axis=1)
    data_input = pd.DataFrame(data_input)
    for i, c in enumerate(data_input.columns):
        data_input[c] = data_input[c].shift(i)

# drop the nan
expected_output = expected_output[to_drop:]
data_input = data_input[to_drop:]

print('Input shape:', data_input.shape)
print('Output shape:', expected_output.shape)
print('Input head: ')
print(data_input.head())
print('Output head: ')
print(expected_output.head())
print('Input tail: ')
print(data_input.tail())
print('Output tail: ')
print(expected_output.tail())

print('Plotting input and expected output')
plt.plot(data_input[0][:10], '.')
plt.plot(expected_output[0][:10], '-')
plt.legend(['Input', 'Expected output'])
plt.title('Input')
plt.show()


def create_model(stateful):
    model = Sequential()
    model.add(LSTM(20,
              input_shape=(lahead, 1),
              batch_size=batch_size,
              stateful=stateful))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

print('Creating Stateful Model...')
model_stateful = create_model(stateful=True)


# split train/test data
def split_data(x, y, ratio=0.8):
    to_train = int(input_len * ratio)
    # tweak to match with batch_size
    to_train -= to_train % batch_size

    x_train = x[:to_train]
    y_train = y[:to_train]
    x_test = x[to_train:]
    y_test = y[to_train:]

    # tweak to match with batch_size
    to_drop = x.shape[0] % batch_size
    if to_drop > 0:
        x_test = x_test[:-1 * to_drop]
        y_test = y_test[:-1 * to_drop]

    # some reshaping
    reshape_3 = lambda x: x.values.reshape((x.shape[0], x.shape[1], 1))
    x_train = reshape_3(x_train)
    x_test = reshape_3(x_test)

    reshape_2 = lambda x: x.values.reshape((x.shape[0], 1))
    y_train = reshape_2(y_train)
    y_test = reshape_2(y_test)

    return (x_train, y_train), (x_test, y_test)


(x_train, y_train), (x_test, y_test) = split_data(data_input, expected_output)
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

print('Training')
for i in range(epochs):
    print('Epoch', i + 1, '/', epochs)
    # Note that the last state for sample i in a batch will
    # be used as initial state for sample i in the next batch.
    # Thus we are simultaneously training on batch_size series with
    # lower resolution than the original series contained in data_input.
    # Each of these series are offset by one step and can be
    # extracted with data_input[i::batch_size].
    model_stateful.fit(x_train,
                       y_train,
                       batch_size=batch_size,
                       epochs=1,
                       verbose=1,
                       validation_data=(x_test, y_test),
                       shuffle=False)
    model_stateful.reset_states()

print('Predicting')
predicted_stateful = model_stateful.predict(x_test, batch_size=batch_size)

print('Creating Stateless Model...')
model_stateless = create_model(stateful=False)

print('Training')
model_stateless.fit(x_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    shuffle=False)

print('Predicting')
predicted_stateless = model_stateless.predict(x_test, batch_size=batch_size)

# ----------------------------

print('Plotting Results')
plt.subplot(3, 1, 1)
plt.plot(y_test)
plt.title('Expected')
plt.subplot(3, 1, 2)
# drop the first "tsteps-1" because it is not possible to predict them
# since the "previous" timesteps to use do not exist
plt.plot((y_test - predicted_stateful).flatten()[tsteps - 1:])
plt.title('Stateful: Expected - Predicted')
plt.subplot(3, 1, 3)
plt.plot((y_test - predicted_stateless).flatten())
plt.title('Stateless: Expected - Predicted')
plt.show()

### Previous Code

In [0]:
def sigma_score(preds, valid_data):
    df_time = valid_data.params['extra_time']
    labels = valid_data.get_label()
    
    assert len(labels) == len(df_time)
    
    x_t = preds * labels

    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()
    return 'sigma_score', score, True


def sigma_score_plain(preds, labels, ds_time):
    x_t = preds * labels
    x_t_sum = x_t.groupby(ds_time.values).sum()
    score = x_t_sum.mean() / x_t_sum.std()
    return score

In [0]:
lgb_params = dict(
    objective = 'regression_l1',
    learning_rate = 0.1,
    num_leaves = 12,
    max_depth = 50,
#     min_data_in_leaf = 1000,
#     min_sum_hessian_in_leaf = 10,
    bagging_fraction = 0.75,
    bagging_freq = 2,
    feature_fraction = 0.5,
    lambda_l1 = 0.0,
    lambda_l2 = .1,
    metric = 'l2', # bbb
    seed = 42
)

# potential area for improvement


class BaseModel():
    def __init__(self):
        self.model = None
        self.xtr = None
        self.ytr = None
        self.xval = None
        self.yval = None
        self.lgb_params = lgb_params
        self.learning_rate_decay_fn = learning_rate_power
        
    
    
    def fit(self, xtr, ytr):
        raise Exception('Not implemented. Override this method.')
    
    
    def predict(self, xval):
        raise Exception('Not implemented. Override this method.')
        


class LGBBasicModel(BaseModel):
    def __init__(self, ban_words=None):
        self.categorical_cols = ['dayofweek', 'month']
        self.lgb_params = lgb_params
        self.remove_list = ['time', 'universe', 'returnsOpenNextMktres10', 'assetCode', 'assetName', 'headline']
        self.ban_words = ban_words
        

    def fit_eval(self, xtr, ytr, xval, yval, num_rounds=None):
        self.fit_full(xtr, ytr, num_rounds)
        return self.predict(xval)
        
        
    def _filter_regex(self, data):
        return data
    
    
    def predict(self, xval):
        ls_to_drop = []
            
        for c in self.remove_list:
            if c in xval.columns:
                ls_to_drop.append(c)
        xval = xval.drop(columns=ls_to_drop)
            
        xval = self._filter_regex(xval)
        
        xval = xval[self.train_cols]
        return self.model.predict(xval)
    
    
    def fit(self, xtr, ytr):
        if self.ban_words is not None:
            drop_list = []
            for col in xtr.columns:
                for ban_word in self.ban_words:
                    if ban_word in col:
                        drop_list.append(col)
                        break
            xtr = xtr.drop(columns=drop_list)
            
        # actual training and validation sets
        xtr_, xval_, ytr_, yval_ = train_test_split(xtr, ytr, test_size=0.2, shuffle=False)
        
        valid_params = {
            'extra_time': xval_['time'].factorize()[0]
        }
        
        ls_to_drop = []
        for col in self.remove_list:
            if col in xtr_.columns:
                ls_to_drop.append(col)
        
        xtr_ = xtr_.drop(columns=ls_to_drop)
        xval_ = xval_.drop(columns=ls_to_drop)
        
        xtr_ = self._filter_regex(xtr_)
        xval_ = self._filter_regex(xval_)
        
        train_cols = xtr_.columns.tolist()
        
        # Note: y data is expected to be a pandas Series, as we will use its group_by function in `sigma_score`
        dtrain = lgb.Dataset(xtr_.values, ytr_, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        dvalid = lgb.Dataset(xval_.values, yval_, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        dvalid.params = valid_params
        evals_result = {}
        m = lgb.train(lgb_params, dtrain, num_boost_round=1000, valid_sets=(dvalid,), valid_names=('valid',), verbose_eval=25,
                      early_stopping_rounds=100, evals_result=evals_result)
        self.train_cols = train_cols
        self.model = m
        self.evals_result = evals_result
    
    
    def fit_full(self, xtr, ytr, num_rounds=None):
        if self.ban_words is not None:
            drop_list = []
            for col in xtr.columns:
                for ban_word in self.ban_words:
                    if ban_word in col:
                        drop_list.append(col)
                        break
            xtr = xtr.drop(columns=drop_list)
        
        if num_rounds is None:
            df_result = pd.DataFrame(self.evals_result['valid'])
            num_rounds = df_result['l2'].argmax()+1
        
        ls_to_drop = []
        for col in self.remove_list:
            if col in xtr.columns:
                ls_to_drop.append(col)
        
        xtr = xtr.drop(columns=ls_to_drop)
        xtr = self._filter_regex(xtr)
        
        train_cols = xtr.columns.tolist()
        dtrain = lgb.Dataset(xtr.values, ytr, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        
        m = lgb.train(self.lgb_params, dtrain, num_boost_round=num_rounds)
        self.train_cols = train_cols
        self.model = m

In [0]:
%%time

def get_stack_data(models):
#     df_xtr = pd.read_hdf('df_roll.h5', 'data')
#     df_xtr = pd.read_pickle('df_roll.p')
#     df_xtr = df_xtr.drop(columns='headline')
    
    global df_xtr
    
    arr_dates = df_xtr['time'].unique()
    dt_start = arr_dates[-(arr_dates.shape[0] // 504) * 504]
    
    df_xtr = df_xtr[df_xtr['time'] >= dt_start]
    
    target = df_xtr['returnsOpenNextMktres10'].copy()
#     stack_data = stack_models(df_xtr, models)
    
    stack_data = df_xtr
    
    stack_data['sentimentPositive_outperform'] = stack_data['sentimentPositive_mean'] - stack_data['sentimentPositive_mean'].mean()
    stack_data['sentimentNegative_outperform'] = stack_data['sentimentNegative_mean'] - stack_data['sentimentNegative_mean'].mean()
#     stack_data['open_close_outperform'] = stack_data['open_close'] - stack_data['open_close'].mean()
#     stack_data['returnsOpenPrevMktres1_outperform'] = stack_data['returnsOpenPrevMktres1'] - stack_data['returnsOpenPrevMktres1'].mean()
#     stack_data['returnsOpenPrevMktres10_outperform'] = stack_data['returnsOpenPrevMktres10'] - stack_data['returnsOpenPrevMktres10'].mean()

#     stack_data.to_hdf('stack_data.h5', 'data')
#     stack_data.to_pickle('stack_data.p')
#     return stack_data, target
    return stack_data

with multiprocessing.Pool(1) as pool:
    df_stack = pool.map(get_stack_data, [None])[0]

In [0]:
def cv_stack(X, model, k=5, seq=True, cv_type='model'):
    if seq:
        k += 1
        breakpoints = [int(float(X.shape[0])*i/float(k)) for i in range(1, k)] + [X.shape[0]]
    else:
        breakpoints = [int(float(X.shape[0])*i/float(k)) for i in range(k)] + [X.shape[0]]
    X = X.set_index('time')
    y = X['returnsOpenNextMktres10']
    X = X.drop(columns=['returnsOpenNextMktres10'])
    res = []
    
    for i in range(len(breakpoints)-1):
        # skip to last round (remove)
        if i+1 < len(breakpoints)-1:
            continue
            
        p1, p2 = breakpoints[i], breakpoints[i+1]
        dt1, dt2 = X.index[p1], X.index[p2] if p2 < X.shape[0] else None
        
        dt1, dt2 = pd.to_datetime(dt1), pd.to_datetime(dt2)
        X_train = X[:dt1 - dt.timedelta(days=1)].reset_index()
        y_train = y[:dt1 - dt.timedelta(days=1)]
        X_test = X[dt1:dt2].reset_index()
        y_test = y[dt1:dt2]
        ds_time = X[dt1:dt2].index
        
        
        print('Fold %d (%s to %s)' % (i+1, dt1, dt2))
        if seq:
            if cv_type == 'model':
                model.fit_full(X_train, y_train, num_rounds=200)
                preds = model.predict(X_test)
            elif cv_type == 'average':
                preds = X_test.mean(axis=1)
            elif cv_type == 'majority':
                preds = []
                for row in X_test.values:
                    if (row > 0).sum() > X.shape[1] / 2:
                        preds.append(row[row>0].mean())
                    else:
                        preds.append(row[row<=0].mean())
            else:
                raise Exception('Unsupported')
            score = sigma_score_plain(np.array(preds), y_test, ds_time)
            res.append(score)
        else:
            raise Exception('Not supported')
            arr_flags = (np.arange(X.shape[0]) < dt1) | (np.arange(X.shape[0]) >= dt2)
            res.append(fit_val
                       (X[arr_flags], y[arr_flags], X.iloc[dt1:dt2], y.iloc[dt1:dt2]))
    return res

In [0]:
%%time

cv_results = cv_stack(df_stack, LGBBasicModel(), cv_type='model')

In [0]:
cv_results